# Dirichlet BDDC Example

In [ ]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

In [ ]:
# setup
p = 2
numbercomponents = 1
dimension = 2
mesh = Mesh2D(1.0, 1.0)
numberelements1d = 4

# operator
function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end
# -- fine
basis = TensorH1LagrangeMacroBasis(p, p + 1, numbercomponents, dimension, numberelements1d);
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
];
outputs = [OperatorField(basis, [EvaluationMode.gradient])];
diffusion = Operator(diffusionweakform, mesh, inputs, outputs);

# bddc smoother
bddc = DirichletBDDC(diffusion)

In [ ]:
# full operator symbols
numberruns = 250
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2

# compute and plot smoothing factor
# setup
maxeigenvalues = zeros(numberruns, numberruns)

# compute
for i in 1:numberruns, j in 1:numberruns
    θ = [
        θ_min + (θ_max - θ_min)*i/numberruns,
        θ_min + (θ_max - θ_min)*j/numberruns
    ]
    if sqrt(abs(θ[1] % 2π)^2 + abs(θ[2] % 2π)^2) > π/128
        A = computesymbols(bddc, θ)
        eigenvalues = [abs(val) for val in eigvals(I-A)]
        maxeigenvalues[i, j] = max(eigenvalues...)
        maxeigenvalue = max(maxeigenvalue, maxeigenvalues[i, j])
    end
end

# plot
println("max eigenvalue: ", maxeigenvalue)
xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
heatmap(
    xrange,
    xlabel="θ/π",
    xtickfont=font(12, "Courier"),
    xrange,
    ylabel="θ/π",
    ytickfont=font(12, "Courier"),
    maxeigenvalues,
    title="Dirichlet BDDC Error Symbol",
    transpose=true,
    aspect_ratio=:equal
)
xlims!(θ_min/π, θ_max/π)
ylims!(θ_min/π, θ_max/π)